In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import json
import regex
import unidecode

In [2]:
# Load data
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/insiders.parquet")
df_in = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/outsiders.parquet")
df_ou = pd.read_parquet(dir_df)
dir_df = Path("C:/Users/josea/Documents/Trabajo/data/metadata/minors.parquet")
df_mi = pd.read_parquet(dir_df)

In [3]:
print(df_in.shape)
print(df_ou.shape)
print(df_mi.shape)

(519520, 195)
(236210, 43)
(1650869, 107)


In [4]:
def unify_colname(col):
    return ".".join([el for el in col if el])


def evaluate(el):
    if not isinstance(el, str):
        return el
    if el[0] == "[" or el[0] == "'":
        return eval(el)
    return el


def fill_na(cell, fill=[]):
    """
    Fill elements in pd.DataFrame with `fill`.
    """
    if hasattr(cell, "__iter__"):
        if isinstance(cell, str):
            if cell == "nan":
                return fill
            return cell
        nas = []
        for el in cell:
            if el == "nan" or pd.isna(el):
                nas.append(True)
            else:
                nas.append(False)
        if all(nas):
            return fill
        return cell
    if pd.isna(cell):
        return fill
    return cell


def process_str(el: str):
    s = regex.sub(r"(^[\W]*)|([\W]*$)|([^\w:/-\s])", "", el).lower().strip()
    s = regex.sub(r"\s+", " ", s)
    return s


def process_cpv(el: str):
    return regex.sub(r"\D", "", el)

In [5]:
# Get general tender info (title, object, winner, etc)
df_in.columns = [unify_colname(c) for c in df_in.columns]
df_ou.columns = [unify_colname(c) for c in df_ou.columns]
df_mi.columns = [unify_colname(c) for c in df_mi.columns]

In [36]:
# Counter(df_in.columns.tolist()+df_ou.columns.tolist()+df_mi.columns.tolist())

In [37]:
# df[
#     "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode"
# ]

In [38]:
# print(sorted(list(set(list(df_in.columns) + list(df_ou.columns) + list(df_mi.columns))), key=len))

In [6]:
use_cols = [
    "id",
    "summary",
    "title",
    "updated",
    # "deleted_on",
    "ContractFolderStatus.ContractFolderID",
    "ContractFolderStatus.ContractFolderStatusCode",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyIdentification.ID",
    "ContractFolderStatus.LocatedContractingParty.Party.PartyName.Name",
    "ContractFolderStatus.ProcurementProject.Name",
    "ContractFolderStatus.ProcurementProject.TypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TaxExclusiveAmount",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.DurationMeasure",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.StartDate",
    "ContractFolderStatus.ProcurementProject.PlannedPeriod.EndDate",
    # "ContractFolderStatus.TenderResult.ResultCode",
    # "ContractFolderStatus.TenderResult.ReceivedTenderQuantity",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
    "ContractFolderStatus.TenderResult.WinningParty.PartyName.Name",
    # "ContractFolderStatus.TenderResult.AwardedTenderedProject.LegalMonetaryTotal.TaxExclusiveAmount",
    # "ContractFolderStatus.TenderingProcess.ProcedureCode",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndDate",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.EndTime",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod.Description",
    # "ContractFolderStatus.TenderingProcess.TenderSubmissionDeadlinePeriod",
    # "ContractFolderStatus.ProcurementProject.SubTypeCode",
    # "ContractFolderStatus.ProcurementProject.BudgetAmount.TotalAmount",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.IdentificationCode",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.Country.Name",
    # "ContractFolderStatus.TenderResult.Description",
    # "ContractFolderStatus.TenderResult.AwardDate",
    # "ContractFolderStatus.TenderResult.StartDate",
    # "ContractFolderStatus.TenderResult.Contract.ID",
    # "ContractFolderStatus.TenderResult.Contract.IssueDate",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.CityName",
    # "ContractFolderStatus.ProcurementProject.RealizedLocation.Address.PostalZone",
    "ContractFolderStatus.TenderingTerms.FundingProgramCode",
    "ContractFolderStatus.TenderingTerms.FundingProgram",
    "ContractFolderStatus.TenderResult.WinningParty.PartyLegalEntity.CompanyTypeCode",
]

In [7]:
df = pd.concat(
    [
        df_in[[c for c in use_cols if c in df_in.columns]],
        df_ou[[c for c in use_cols if c in df_ou.columns]],
        # df_mi[[c for c in use_cols if c in df_mi.columns]],
    ]
)
index_names = df.index.names
df.reset_index(inplace=True)
df["identifier"] = df[index_names].astype(str).agg("/".join, axis=1)
# df.drop(index_names, inplace=True, axis=1)
df.set_index("identifier", inplace=True)
df = df.applymap(fill_na, fill=np.nan)
df.shape

(755730, 24)

In [41]:
# with open(r"C:\Users\josea\Downloads\genCat_Junio_2023.json", "r") as f:
#     gencat = pd.json_normalize(json.load(f))

In [8]:
# emp2 = pd.read_csv(r"C:\Users\josea\Downloads\empresas2.csv")
emp = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")

C:\Users\josea\AppData\Local\Temp\ipykernel_38872\2519398558.py:2: DtypeWarning: Columns (27,43,52,53,55,56,57,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  emp = pd.read_csv(r"C:\Users\josea\Downloads\empresas.csv")


In [9]:
rename_columns = {
    "title": "title",
    "summary": "summary",
    "ContractFolderStatus.ContractFolderID": "id",
    "ContractFolderStatus.ProcurementProject.Name": "project",
    "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID": "winner",
    "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode": "cpv",
    "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity": "location",
    "denominacio": "title",
    "objecte_contracte": "summary",
    "codi_expedient": "id",
    "codi_cpv": "cpv",
}

In [10]:
party_ids = (
    df[
        [
            "title",
            "summary",
            "ContractFolderStatus.ContractFolderID",
            # "ContractFolderStatus.ProcurementProject.Name",
            # "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
            "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
            "ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentity",
        ]
    ]
    .rename(columns=rename_columns)
    .dropna(how="all")
    .explode("cpv")
    # .explode("winner")
)
party_ids["id"] = party_ids["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
# party_ids["winner"] = party_ids["winner"].apply(evaluate)
party_ids["cpv"] = party_ids["cpv"].apply(evaluate)
party_ids = party_ids.explode("cpv")  # .explode("winner")
party_ids["cpv"] = party_ids["cpv"].astype(str).apply(process_cpv)
party_ids["location"] = party_ids["location"].apply(
    lambda x: unidecode.unidecode(x.lower()) if not pd.isna(x) else np.nan
)
party_ids = (
    party_ids.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
)
party_ids["cpv_div"] = party_ids["cpv"].apply(
    lambda x: x[:2] if not pd.isna(x) else np.nan
)

In [11]:
agg = (
    party_ids[
        party_ids["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .groupby(["id"])
    .agg(list)
)
valid_agg = agg[agg["title"].apply(lambda x: len(set(x)) == 1)]
valid_agg = valid_agg.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()
print(valid_agg.shape)
display(valid_agg.head())

In [38]:
party_ids["id_2"] = party_ids["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"\W", "", x)) if not pd.isna(x) else np.nan
)
agg2 = (
    party_ids[
        party_ids["location"].str.contains(
            "|".join(
                [
                    "nan",
                    "espana",
                    "cataluna",
                    "catalunya",
                    "barcelona",
                    "tarragona",
                    "girona",
                    "gerona",
                    "lleida",
                    "lerida",
                ]
            )
        )
    ]
    .groupby(["id_2"])
    .agg(list)
)
valid_agg2 = agg2[agg2["title"].apply(lambda x: len(set(x)) == 1)]
valid_agg2 = valid_agg2.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()
print(valid_agg2.shape)
display(valid_agg2.head())

(242772, 7)


,id_2,title,summary,id,cpv,location,cpv_div
0,000000,licitació de prova sobre digital 00/0000 - pro...,id licitación: 00/0000 órgano de contratación:...,00-0000,791000000,nan,79
1,0000000000009322021,suministro de medios de contraste radiológicos...,id licitación: 000000000000932/2021 órgano de ...,000000000000932-2021,331000000,nan,33
2,00000011117,n1333071/0311 servicios para la redacción del ...,id licitación: 000000111/17 órgano de contrata...,000000111-17,71354400,espana,71
3,00000011217,n1339060/0311 servicios para la redacción del ...,id licitación: 000000112/17 órgano de contrata...,000000112-17,71356400,espana,71
4,00000013201701451,suministro de energía eléctrica para el ceremede,id licitación: 0000-001-3-2017-01451órgano de ...,0000-001-3-2017-01451,311540000,nan,31


In [47]:
# agg2 = (
#     party_ids[
#         ~party_ids["location"].str.contains(
#             "|".join(
#                 [
#                     "nan",
#                     "espana",
#                     "cataluna",
#                     "catalunya",
#                     "barcelona",
#                     "tarragona",
#                     "girona",
#                     "gerona",
#                     "lleida",
#                     "lerida",
#                 ]
#             )
#         )
#     ]
#     .groupby(["id"])
#     .agg(list)
# )
# valid_agg2 = agg2[agg2["title"].apply(lambda x: len(set(x)) == 1)]
# valid_agg2 = valid_agg2.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()

In [48]:
# print(valid_agg2.shape)
# display(valid_agg2.head())

In [49]:
# gencat_ids = (
#     gencat[
#         [
#             "title",
#             "ContractFolderStatus.ContractFolderID",
#             "ContractFolderStatus.ProcurementProject.Name",
#             "ContractFolderStatus.TenderResult.WinningParty.PartyIdentification.ID",
#             "ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode",
#         ]
#     ]
#     .rename(columns=rename_columns)
#     .applymap(str.lower)
# )

In [13]:
emp_ids = (
    emp[
        [
            "denominacio",
            "objecte_contracte",
            "codi_expedient",
            "codi_cpv",
        ]
    ]
    .rename(columns=rename_columns)
    .astype(str)
    .applymap(process_str)
)
emp_ids["id"] = emp_ids["id"].apply(
    lambda x: unidecode.unidecode(regex.sub(r"[^\p{L}\d]+", "-", x))
    if not pd.isna(x)
    else np.nan
)
emp_ids["cpv"] = emp_ids["cpv"].astype(str).apply(process_cpv)
emp_ids = emp_ids.astype(str).applymap(process_str).replace({"": np.nan, "0": np.nan})
emp_ids["cpv_div"] = emp_ids["cpv"].apply(lambda x: x[:2] if not pd.isna(x) else np.nan)

In [60]:
valid_emp = emp_ids.groupby("id").agg(list)
valid_emp = valid_emp[valid_emp["title"].apply(lambda x: len(set(x)) == 1)]
valid_emp = valid_emp.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()
print(len(valid_emp))
display(valid_emp.head())

227651


,id,title,summary,cpv,cpv_div,id_2
0,0-2020-5-2020,rehabilitació de casa de poblet,rehabilitació de casa de poblet,454541005,45,0202052020
1,0-s-04-2023,gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,79,0s042023
2,00-0000,licitació de prova sobre digital,licitació de prova sobre digital 00/0000 - pro...,791000005,79,000000
3,000,concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,NaN,000
4,000-expedient-2020-034,000 expedient 2020 034 - licitació de la contr...,en el marc duna creixent proliferació quant a ...,926220007,92,000expedient2020034


In [61]:
# emp_ids["id_2"] = emp_ids["id"].apply(
#     lambda x: unidecode.unidecode(regex.sub(r"\W", "", x)) if not pd.isna(x) else np.nan
# )
# valid_emp2 = emp_ids.groupby("id_2").agg(list)
# valid_emp2 = valid_emp2[valid_emp2["title"].apply(lambda x: len(set(x)) == 1)]
# valid_emp2 = valid_emp2.applymap(lambda x: Counter(x).most_common()[0][0]).reset_index()
# print(len(valid_emp2))
# display(valid_emp2.head())

In [66]:
matched_all = pd.merge(
    valid_agg,
    valid_emp,
    how="inner",
    left_on=["id"],
    right_on=["id"],
)
matched_all = matched_all[
    [
        "id",
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
    ]
]
matched_all["title"] = dict()
matched_all["title"].update(
    matched_all["title_x"].apply(lambda x: Counter([x]) if not pd.isna(x) else np.nan)
)
matched_all["title"].update(
    matched_all["title_y"].apply(lambda x: Counter([x]) if not pd.isna(x) else np.nan)
)
matched_all["cpv_div"] = dict()
matched_all["cpv_div"].update(
    matched_all["cpv_div_x"].apply(lambda x: Counter([x]) if not pd.isna(x) else np.nan)
)
matched_all["cpv_div"].update(
    matched_all["cpv_div_y"].apply(lambda x: Counter([x]) if not pd.isna(x) else np.nan)
)
matched_all = matched_all[["id", "title", "cpv_div"]]
print(len(matched_all))
display(matched_all.head())

83775


,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,{'licitació de prova sobre digital': 1},{'79': 1}
2,000-expedient-2020-034,{'000 expedient 2020 034 - licitació de la con...,{'92': 1}
3,000-expedient-2021-266,{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa de subministrament dai...,{'45': 1}


In [67]:
matched_aux = pd.merge(
    valid_agg,
    valid_emp,
    how="inner",
    left_on=[
        "title",
        "cpv_div",
    ],
    right_on=[
        "title",
        "cpv_div",
    ],
)
matched_aux = matched_aux[
    [
        "id_x",
        "id_y",
        "title",
        "cpv_div",
    ]
]
matched_aux = matched_aux[matched_aux["id_x"] != matched_aux["id_y"]]
m_x = matched_aux.groupby("id_x").agg(Counter).drop("id_y", axis=1)
m_x.index.name = "id"
m_y = matched_aux.groupby("id_y").agg(Counter).drop("id_x", axis=1)
m_y.index.name = "id"
matched_aux = pd.merge(
    m_x,
    m_y,
    how="outer",
    left_on="id",
    right_on="id",
).reset_index()
matched_aux["title"] = dict()
matched_aux["title"].update(matched_aux["title_x"])
matched_aux["title"].update(matched_aux["title_y"])
matched_aux["cpv_div"] = dict()
matched_aux["cpv_div"].update(matched_aux["cpv_div_x"])
matched_aux["cpv_div"].update(matched_aux["cpv_div_y"])
matched_aux = matched_aux[["id", "title", "cpv_div"]]
print(len(matched_aux))
display(matched_aux.head())

6958


,id,title,cpv_div
0,001-21000133,{'servei dassessorament mediació i administrac...,{'66': 1}
1,001-21000518,{'realització de les adaptacions de la creativ...,{'79': 1}
2,001-21002534,{'servei dassessorament mediació i administrac...,{'66': 1}
3,001-21003081,{'subministrament de material de conservació i...,{'30': 1}
4,001-22002143,{'realització de les adaptacions de la creativ...,{'79': 1}


In [68]:
matched_final = pd.merge(
    matched_all,
    matched_aux,
    how="outer",
    left_on="id",
    right_on="id",
)
matched_final["title"] = dict()
matched_final["title"].update(matched_final["title_x"])
matched_final["title"].update(matched_final["title_y"])
matched_final["cpv_div"] = dict()
matched_final["cpv_div"].update(matched_final["cpv_div_x"])
matched_final["cpv_div"].update(matched_final["cpv_div_y"])
matched_final = matched_final[["id", "title", "cpv_div"]]

In [71]:
valid_emp[~valid_emp["id"].isin(matched_final["id"])]

,id,title,summary,cpv,cpv_div,id_2
1,0-s-04-2023,gestió duna installació itinerant sobre la pro...,gestió duna installació itinerant que parli de...,794210001,79,0s042023
3,000,concessió dús privatiu per a la installació de...,concessió dús privatiu per a la installació de...,NaN,NaN,000
6,0000-00,prova de leina sobre digital 20 contracte fict...,contractació fícticia per provar leina sobre d...,926200003,92,000000
7,00000-2015,contracte menor prova,prova,031161008,03,000002015
8,000000000873,contractes menors fundacio mies van der rohe 4...,manteniment muntacarregues pavelló mies,507500007,50,000000000873
...,...,...,...,...,...,...
227624,xacon03-2022,campanya de comunicació estiu 2022,nan,NaN,NaN,xacon032022
227634,xx-xx,acord marc del subministrament dantiseptics i ...,us comuniquem que des del consorci de salut i ...,NaN,NaN,xxxx
227635,xxx,xxx_test,test,450000007,45,xxx
227649,zls23-0084,2023 1r trimestre contractes menors associació...,traduccions,795300008,79,zls230084


In [53]:
matched_final["title"].apply(lambda x: sum(x.values())).sum()

126602

In [70]:
matched_final

,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,{'licitació de prova sobre digital': 1},{'79': 1}
2,000-expedient-2020-034,{'000 expedient 2020 034 - licitació de la con...,{'92': 1}
3,000-expedient-2021-266,{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa de subministrament dai...,{'45': 1}
...,...,...,...
86743,tnc202000001,{'contractació de la conquesta del pol sud scp...,{'92': 1}
86744,uvic-ucc-201704,{'servei postal dels diversos centres de la un...,{'60': 1}
86745,x2021000479,{'subministrament de material informàtic': 2},{'30': 2}
86746,x2023000547,{'contracte de serveis de neteja dels edificis...,{'90': 1}


In [23]:
matched_final

,id,title,cpv_div
0,0-2020-5-2020,{'rehabilitació de casa de poblet': 1},{'45': 1}
1,00-0000,{'licitació de prova sobre digital': 1},{'79': 1}
2,000-expedient-2020-034,{'000 expedient 2020 034 - licitació de la con...,{'92': 1}
3,000-expedient-2021-266,{'000 expedient 2021 266 - expedient de contra...,{'34': 1}
4,00000057-2022,{'renovació de la xarxa de subministrament dai...,{'45': 1}
...,...,...,...
86743,tnc202000001,{'contractació de la conquesta del pol sud scp...,{'92': 1}
86744,uvic-ucc-201704,{'servei postal dels diversos centres de la un...,{'60': 1}
86745,x2021000479,{'subministrament de material informàtic': 2},{'30': 2}
86746,x2023000547,{'contracte de serveis de neteja dels edificis...,{'90': 1}


In [30]:
# matched_final[matched_final["title"].apply(lambda x: len(x.values()))>1].values

In [60]:
len(matched_final) / len(valid_emp), matched_final["title"].apply(
    lambda x: sum(x.values())
).sum() / len(valid_emp)

(0.006997553272333528, 0.007028302093994755)

In [62]:
len(valid_emp) * 0.5485282296146294

124873.0

In [24]:
len(matched_final) / len(valid_emp), matched_final["title"].apply(
    lambda x: sum(x.values())
).sum() / len(valid_emp)

(0.38105696878116063, 0.5485282296146294)

In [33]:
# Get same title and cpv, but different id
matched_tmp = pd.merge(
    valid_agg,
    valid_emp,
    how="outer",
    left_on="id",
    right_on="id",
)
matched_tmp = matched_tmp[
    [
        "id",
        "title_x",
        "title_y",
        "cpv_div_x",
        "cpv_div_y",
        "location",
        "summary_x",
        "summary_y",
    ]
]
rest = matched_tmp.groupby(["title_x", "title_y", "cpv_div_x", "cpv_div_y"]).agg(list)
rest[rest["id"].apply(len) > 1]

,,,,id,location,summary_x,summary_y
title_x,title_y,cpv_div_x,cpv_div_y,,,,
6318 - 45800 obres adequació interior ot terrassa,6318 - 45800 obres adequació interior ot terrassa,45,45,"[soc-2021-132, soc-2021-631]","[nan, nan]",[id licitación: soc-2021-132 órgano de contrat...,[6318 - 45800 obres adequació interior ot terr...
acord marc de lhomologació de proveïdors per als projectes de desenvolupament daplicacions de la uoc,acord marc de lhomologació deproveïdors per als projectes de desenvolupament daplicacions de la uoc,72,72,"[der00018-2018, der00019-2018]","[nan, nan]",[id licitación: der00018/2018 órgano de contra...,[acord marc de lhomologació de proveïdors per ...
acord marc del subministrament de material fungible doftalmologia per processos,acord marc del subministrament de material fungible doftalmologia per processos,33,33,"[csc-m-6-19, csc-m-6-19-vo]","[nan, nan]",[id licitación: csc m 6/19 órgano de contratac...,[acord marc del subministrament de material fu...
acord marc del subministrament de medicaments antiretrovirals,acord marc del subministrament de medicaments antiretrovirals,33,33,"[csc-f-13-22, csc-f-4-19, csc-f-9-18]","[nan, nan, nan]",[id licitación: csc f 13/22 órgano de contrata...,[acord marc del subministrament de medicaments...
acord marc del subministrament de paper,acord marc del subministrament de paper,30,30,"[csc-m-6-21, csc-m-7-21]","[nan, nan]",[id licitación: csc m 6/21 órgano de contratac...,"[acord marc del subministrament de paper, acor..."
...,...,...,...,...,...,...,...
urbanització dels carrers verge del sòl del pont i call de la capella - fase 1,contracte dobres per a la urbanització dels carrers verge del sòl del pont i call de la capella - fase 1,45,45,"[x2021001952, x2021002381]","[nan, nan]",[id licitación: x2021001952 órgano de contrata...,[urbanització dels carrers verge del sòl del p...
és el subministrament de gasoil dautomoció gasoil a per als vehicles de lajuntament dulldecona el consum anual estimat a títol orientatiu és de 25000 litres anuals lajuntament efectuarà comandes de combustible en funció de les seves necessitats durant la vigència del contracte fins arribar a limport màxim previst en lapartat 33 del plec de clàusules administratives,contractació del subministrament de gasoil dautomoció,91,09,"[1-10-20-9, 4315670005-2022-0000823]","[nan, nan]",[id licitación: 1-10-20-9 órgano de contrataci...,[és el subministrament de gasoil dautomoció ga...
és lobjecte del present contracte la dinamització de lequipament juvenil tangram de santa margarida i els monjos,contracte de servei de dinamització de lequipament juvenil tangram al municipi de santa margarida i els monjos,80,80,"[2022001616, x2020001122]","[nan, nan]",[id licitación: 2022001616 órgano de contratac...,[és lobjecte del present contracte la dinamitz...


In [34]:
list(rest[rest["id"].apply(len) > 1]["id"].items())

[(('6318 - 45800 obres adequació interior ot terrassa',
   '6318 - 45800 obres adequació interior ot terrassa',
   '45',
   '45'),
  ['soc-2021-132', 'soc-2021-631']),
 (('acord marc de lhomologació de proveïdors per als projectes de desenvolupament daplicacions de la uoc',
   'acord marc de lhomologació deproveïdors per als projectes de desenvolupament daplicacions de la uoc',
   '72',
   '72'),
  ['der00018-2018', 'der00019-2018']),
 (('acord marc del subministrament de material fungible doftalmologia per processos',
   'acord marc del subministrament de material fungible doftalmologia per processos',
   '33',
   '33'),
  ['csc-m-6-19', 'csc-m-6-19-vo']),
 (('acord marc del subministrament de medicaments antiretrovirals',
   'acord marc del subministrament de medicaments antiretrovirals',
   '33',
   '33'),
  ['csc-f-13-22', 'csc-f-4-19', 'csc-f-9-18']),
 (('acord marc del subministrament de paper',
   'acord marc del subministrament de paper',
   '30',
   '30'),
  ['csc-m-6-21', 'cs

In [225]:
matched_tmp[matched_tmp["id_x"] != matched_tmp["id_y"]].head().values

array([['001-21000133', '001-21002534',
        'servei dassessorament mediació i administració de les pòlisses dassegurances subscrites o que hagi de subscriure lajuntament de barcelona',
        '66', 'nan',
        'id licitación: 001/21000133 órgano de contratación: ajuntament de barcelona importe: 25108350 estado: adjudicada',
        'servei dassessorament mediació i administració de les pòlisses dassegurances subscrites o que hagi de subscriure lajuntament de barcelona'],
       ['001-21002534', '001-21000133',
        'servei dassessorament mediació i administració de les pòlisses dassegurances subscrites o que hagi de subscriure lajuntament de barcelona',
        '66', 'nan',
        'id licitación: 001/21002534 órgano de contratación: ajuntament de barcelona importe: 3500000 estado: resuelta',
        'servei dassessorament mediació i administració de les pòlisses dassegurances subscrites o que hagi de subscriure lajuntament de barcelona'],
       ['001-21000518', '001_220021

In [211]:
matched_all["location"].value_counts()

location
nan          78683
espana         291
girona           7
tarragona        6
barcelona        3
Name: count, dtype: int64

In [35]:
matched_1 = pd.merge(
    valid_agg,
    valid_emp,
    how="inner",
    left_on=[
        "id",
        "title",
    ],
    right_on=[
        "id",
        "title",
    ],
)
matched_1 = matched_1[
    [
        "id",
        # "id_x",
        # "id_y",
        "title",
        # "title_x",
        # "title_y",
        "cpv_div_x",
        "cpv_div_y",
        # "cpv_x",
        # "cpv_y",
        "location",
        "summary_x",
        "summary_y",
    ]
]

In [36]:
matched_2 = pd.merge(
    # gencat_ids,
    valid_agg,  # [((a["title"].apply(lambda x: len(x) == 1)))],# & (a["cpv_div"].apply(lambda x: len(x) == 1)))],  # [["id", "cpv", "title", "summary", "project"]],
    valid_emp,  # [((b["title"].apply(lambda x: len(x) == 1)))],# & (b["cpv_div"].apply(lambda x: len(x) == 1)))],
    how="inner",
    left_on=[
        "id",
        # "title",
        "cpv_div",
    ],
    right_on=[
        "id",
        # "title",
        "cpv_div",
    ],
)
matched_2 = matched_2[
    [
        "id",
        # "id_x",
        # "id_y",
        # "title",
        "title_x",
        "title_y",
        "cpv_div",
        # "cpv_div_x",
        # "cpv_div_y",
        # "cpv_x",
        # "cpv_y",
        "location",
        "summary_x",
        "summary_y",
    ]
]

In [37]:
combined = matched_all[
    matched_all["id"].isin((set(matched_1["id"]) | set(matched_2["id"])))
]
combined.shape

(70028, 8)

In [47]:
m_tmp = matched_tmp.groupby("id").agg(list)

In [59]:
type(m_tmp["title_x"].values[1][0])

float

In [63]:
m_tmp[m_tmp[["title_x", "title_y"]].applymap(lambda x: not pd.isna(x[0])).all(axis=1)]

,title_x,title_y,cpv_div_x,cpv_div_y,location,summary_x,summary_y
id,,,,,,,
0-2020-5-2020,[rehabilitació de casa de poblet],[rehabilitació de casa de poblet],[45],[45],[nan],[id licitación: 0/2020-5/2020 órgano de contra...,[rehabilitació de casa de poblet]
00-0000,[licitació de prova sobre digital 00/0000 - pr...,[licitació de prova sobre digital],[79],[79],[nan],[id licitación: 00/0000 órgano de contratación...,[licitació de prova sobre digital 00/0000 - pr...
000-expedient-2020-034,[en el marc duna creixent proliferació quant a...,[000 expedient 2020 034 - licitació de la cont...,[92],[92],[nan],[id licitación: 000 expedient 2020 034 órgano ...,[en el marc duna creixent proliferació quant a...
000-expedient-2021-266,[el present expedient té per objecte el submin...,[000 expedient 2021 266 - expedient de contrac...,[34],[34],[nan],[id licitación: 000 expedient 2021 266 órgano ...,[el present expedient té per objecte el submin...
00000057-2022,[renovació de la xarxa daigua potable i implan...,[renovació de la xarxa de subministrament daig...,[45],[45],[nan],[id licitación: 00000057/2022 órgano de contra...,[renovació de la xarxa daigua potable i implan...
...,...,...,...,...,...,...,...
z2016000453,[desenvolupar una actuacio per prevenir els pr...,[projecte dobres de drenatge de la zona del ce...,[45],[45],[nan],[id licitación: z2016000453 órgano de contrata...,[desenvolupar una actuacio per prevenir els pr...
z8377,[obres de la xarxa del tub verd des de la roto...,[obres de la xarxa del tub verd des de la roto...,[90],[09],[nan],[id licitación: z8377 órgano de contratación: ...,[obres de la xarxa del tub verd des de la roto...
z8489,[subministrament i installació de lescomesa de...,[subministrament i installació de lescomesa de...,[51],[51],[nan],[id licitación: z8489 órgano de contratación: ...,[subministrament i installació de lescomesa de...


In [39]:
matched_all.shape, matched_tmp.shape, matched_1.shape, matched_2.shape

((79771, 8), (396858, 8), (25047, 7), (65591, 7))

In [198]:
print(
    matched_all.loc[
        matched_all.index.difference(combined.index)
    ].values  # [["title_x", "title_y"]]
)

[['000050-2018'
  'adquisió dun vehicle amb destí als serveis municipals contracte basat en lacord marc de mobilitat sostenible amb destinació a les entitats locals de catalunya exp 201602'
  'adquisició dun vehicle pels serveis municipals' ... 'nan'
  'id licitación: 000050-2018 órgano de contratación: ajuntament durús importe: 2267952 estado: adjudicada'
  'adquisió dun vehicle amb destí als serveis municipals contracte basat en lacord marc de mobilitat sostenible amb destinació a les entitats locals de catalunya exp 201602']
 ['0001-2020'
  'lacord marc preveu homologar un màxim de sis empreses per tal que puguin subministrar farina panificable a les diferents fleques dels centres penitenciaris de catalunya mitjançant contractes basats que tindran una durada aproximada dentre sis i dotze mesos aquesta farina sutilitza per fer el pà que forma part dels menús diaris dels interns dels diferents centres penitenciaris de catalunya lacord marc tindrà una durada dun any podent-se prorrogar

In [35]:
matched_all.shape, matched2.shape

((98476, 11), (90996, 11))

In [107]:
matched

,cpv_div_x,cpv_div_y,cpv_x,cpv_y,location,project,summary_x,summary_y,title_x,title_y,winner
id,,,,,,,,,,,
0/2020-5/2020,{'45': 1},{'45': 1},{'454541000': 1},{'454541005': 1},{'nan': 1},{'rehabilitació de casa de poblet': 1},{'id licitación: 0/2020-5/2020 órgano de contr...,{'rehabilitació de casa de poblet': 1},{'rehabilitació de casa de poblet': 1},{'rehabilitació de casa de poblet': 1},{'b25768342': 1}
00/0000,{'79': 1},{'79': 1},{'791000000': 1},{'791000005': 1},{'nan': 1},{'licitació de prova sobre digital 00/0000 - p...,{'id licitación: 00/0000 órgano de contratació...,{'licitació de prova sobre digital 00/0000 - p...,{'licitació de prova sobre digital 00/0000 - p...,{'licitació de prova sobre digital': 1},{'nan': 1}
000 expedient 2020 034,{'92': 1},{'92': 1},{'926220000': 1},{'926220007': 1},{'nan': 1},{'en el marc duna creixent proliferació quant ...,{'id licitación: 000 expedient 2020 034 órgano...,{'en el marc duna creixent proliferació quant ...,{'en el marc duna creixent proliferació quant ...,{'000 expedient 2020 034 - licitació de la con...,{'nan': 1}
000 expedient 2021 266,{'34': 1},{'34': 1},{'341000000': 1},{'341000008': 1},{'nan': 1},{'el present expedient té per objecte el submi...,{'id licitación: 000 expedient 2021 266 órgano...,{'el present expedient té per objecte el submi...,{'el present expedient té per objecte el submi...,{'000 expedient 2021 266 - expedient de contra...,{'b25620089': 1}
00000057/2022,{'45': 1},{'45': 1},{'452321500': 1},{'452321508': 1},{'nan': 1},{'renovació de la xarxa daigua potable i impla...,{'id licitación: 00000057/2022 órgano de contr...,{'renovació de la xarxa daigua potable i impla...,{'renovació de la xarxa daigua potable i impla...,{'renovació de la xarxa de subministrament dai...,{'nan': 1}
...,...,...,...,...,...,...,...,...,...,...,...
z2016000453,{'45': 1},{'45': 1},{'452324510': 1},{'452324518': 1},{'nan': 1},{'desenvolupar una actuacio per prevenir els p...,{'id licitación: z2016000453 órgano de contrat...,{'desenvolupar una actuacio per prevenir els p...,{'desenvolupar una actuacio per prevenir els p...,{'projecte dobres de drenatge de la zona del c...,{'b55101059': 1}
z8377,{'90': 1},{'09': 1},{'90000000': 1},{'090000003': 1},{'nan': 1},{'obres de la xarxa del tub verd des de la rot...,{'id licitación: z8377 órgano de contratación:...,{'obres de la xarxa del tub verd des de la rot...,{'obres de la xarxa del tub verd des de la rot...,{'obres de la xarxa del tub verd des de la rot...,{'a08112716': 1}
z8489,{'51': 1},{'51': 1},{'515000000': 1},{'515000007': 1},{'nan': 1},{'subministrament i installació de lescomesa d...,{'id licitación: z8489 órgano de contratación:...,{'subministrament i installació de lescomesa d...,{'subministrament i installació de lescomesa d...,{'subministrament i installació de lescomesa d...,{'a08112716': 1}


In [32]:
matched["cpv_x"].values

array(['454541000', '791000000', '926220000', ..., '515000000',
       '317123310', '452200000'], dtype=object)

In [35]:
dup_id = matched[matched["id"].duplicated(keep=False)]

In [36]:
sub = dup_id[dup_id["id"] == "0001/2019"]
sub["title_x"] == sub["title_y"]

10     False
11     False
12     False
13     False
14     False
       ...  
580    False
581    False
582    False
583    False
584    False
Length: 575, dtype: bool

In [37]:
sub.groupby("id").agg(Counter).values.tolist()

[[Counter({'45': 500, '39': 25, '55': 25, '90': 25}),
  Counter({'45': 506, '90': 23, '39': 23, nan: 23}),
  Counter({'39000000': 25,
           '45000000': 25,
           '450000000': 25,
           '452321300': 25,
           '45310000': 25,
           '45314000': 25,
           '45330000': 25,
           '45331000': 25,
           '45332300': 25,
           '45332400': 25,
           '45333000': 25,
           '45400000': 25,
           '45420000': 25,
           '45421132': 25,
           '45421143': 25,
           '45421151': 25,
           '45421152': 25,
           '45421160': 25,
           '45430000': 25,
           '45441000': 25,
           '45442100': 25,
           '550000000': 25,
           '90910000': 25}),
  Counter({'454211603': 46,
           '453300009': 46,
           '453100003': 46,
           '454300000': 46,
           '909100009': 23,
           '450000007': 23,
           '454211517': 23,
           '390000002': 23,
           '453323006': 23,
           '453

In [60]:
sub.values

array([['39000000', '909100009', '0001/2019', ...,
        'homologació dempreses i industrials per obres de reparació mnteniment i reforma dels habitatges i altres immobles gestionats per habitatge terrassa',
        'acord marc per a lhomologació dempreses i industrials per obres de reparació mnteniment i reforma dels habitatges i altres immobles gestionats per habitatge terrassa',
        'b60188620'],
       ['39000000', '450000007', '0001/2019', ...,
        'homologació dempreses i industrials per obres de reparació mnteniment i reforma dels habitatges i altres immobles gestionats per habitatge terrassa',
        'acord marc per a lhomologació dempreses i industrials per obres de reparació mnteniment i reforma dels habitatges i altres immobles gestionats per habitatge terrassa',
        'b60188620'],
       ['39000000', '454211603', '0001/2019', ...,
        'homologació dempreses i industrials per obres de reparació mnteniment i reforma dels habitatges i altres immobles gestiona

In [30]:
matched[matched["id"].duplicated(keep=False)]

,cpv_x,cpv_y,id,location,project,summary_x,summary_y,title_x,title_y,winner
10,39000000,909100009,0001/2019,nan,homologació dempreses i industrials per obres ...,id licitación: 0001/2019 órgano de contratació...,homologació dempreses i industrials per obres ...,homologació dempreses i industrials per obres ...,acord marc per a lhomologació dempreses i indu...,b60188620
11,39000000,450000007,0001/2019,nan,homologació dempreses i industrials per obres ...,id licitación: 0001/2019 órgano de contratació...,homologació dempreses i industrials per obres ...,homologació dempreses i industrials per obres ...,acord marc per a lhomologació dempreses i indu...,b60188620
12,39000000,454211603,0001/2019,nan,homologació dempreses i industrials per obres ...,id licitación: 0001/2019 órgano de contratació...,homologació dempreses i industrials per obres ...,homologació dempreses i industrials per obres ...,acord marc per a lhomologació dempreses i indu...,b60188620
13,39000000,454211603,0001/2019,nan,homologació dempreses i industrials per obres ...,id licitación: 0001/2019 órgano de contratació...,homologació dempreses i industrials per obres ...,homologació dempreses i industrials per obres ...,acord marc per a lhomologació dempreses i indu...,b60188620
14,39000000,454211517,0001/2019,nan,homologació dempreses i industrials per obres ...,id licitación: 0001/2019 órgano de contratació...,homologació dempreses i industrials per obres ...,homologació dempreses i industrials per obres ...,acord marc per a lhomologació dempreses i indu...,b60188620
...,...,...,...,...,...,...,...,...,...,...
1733745,601300000,601300008,xxx-xx,nan,contracte del servei de transport escolar de l...,id licitación: xxx-xx órgano de contratación: ...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,nan
1733746,601300000,601300008,xxx-xx,nan,contracte del servei de transport escolar de l...,id licitación: xxx-xx órgano de contratación: ...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,nan
1733747,601300000,601300008,xxx-xx,nan,contracte del servei de transport escolar de l...,id licitación: xxx-xx órgano de contratación: ...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,nan
1733748,601300000,601300008,xxx-xx,nan,contracte del servei de transport escolar de l...,id licitación: xxx-xx órgano de contratación: ...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,contracte del servei de transport escolar de l...,nan
